# objectives - get basic model tested after variance inflation factor used for feature selection

- check feature importance afterwards, 

- set up for advanced model tuning 

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns

In [ ]:
expected_model_version = '1.0'
model_path = 'models/11th_grade_scoring_model.pkl'
if os.path.exists(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    if model.version != expected_model_version:
        print("Expected model version doesn't match version loaded")
    if model.sklearn_version != sklearn_version:
        print("Warning: model created under different sklearn version")
else:
    print("Expected model not found")

In [ ]:
data = pd.read_csv('../../data/model/final_df.csv')

In [ ]:
mills = data[data.school_name == 'Mills High School']

capuchino = data[data['School Name'] == 'Cap ']

In [ ]:
cap.T 

In [ ]:
def predict_change( model, school,features, deltas):
    
    adj_school = school.copy()
    for f,d in zip(features, deltas):
        adj_school[f] += d
    return model.predict(adj_school).item() - model.predict(school).item()

## refit model with all data 


plot compare key features to be adjusted 

use predict increase to generate alternative scores due to change in model features 

graph changes in scores due to adjusted features 

